In [1]:
!head -10 ml-1m/ratings.dat


1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968
1::3408::4::978300275
1::2355::5::978824291
1::1197::3::978302268
1::1287::5::978302039
1::2804::5::978300719
1::594::4::978302268
1::919::4::978301368


In [2]:
import pandas as pd
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter="::", engine="python", header=None)

In [3]:
ratings.columns = ["UserID::MovieID::Rating::Timestamp".split("::")]
ratings.head()

UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291

In [8]:
pd.read_csv?

In [4]:
ratings.isnull().sum()

UserID       0
MovieID      0
Rating       0
Timestamp    0
dtype: int64

In [5]:
import numpy as np
np.unique(ratings['Rating'])

array([1, 2, 3, 4, 5])

* Do we have any duplicate ratings, i.e., a (userid, movieid) combination that is repeated more than once?

In [7]:
user_movie_pairs = zip(ratings['UserID'].values, ratings['MovieID'].values)

In [8]:
print len(user_movie_pairs), len(set(user_movie_pairs))

1000209 1000209


In [9]:
from scipy.sparse import csr_matrix
row = ratings['MovieID'].values
col = ratings['UserID'].values
data = ratings['Rating'].values
ratings_array = csr_matrix((data, (row, col)))



In [10]:
ratings_array.shape

(3953, 6041)

In [11]:
np.sum(ratings_array != 0)

1000209

In [12]:
import numpy as np
#X = np.array([[1,1], [2, 1], [3, 1.2], [4, 1], [5, 0.8], [6, 1]])
from sklearn.decomposition import NMF
model = NMF(n_components=5, init='random', random_state=0)
W = model.fit_transform(ratings_array) 
H = model.components_



In [13]:
H.shape


(5, 6041)

In [53]:
model.n_components_

5

In [18]:
X = ratings_array

In [78]:
def trace_dot(X, Y):
    """Trace of np.dot(X, Y.T)."""
    return np.dot(X.ravel(), Y.ravel())
norm_X = np.dot(X.data, X.data)
norm_WH = trace_dot(np.dot(np.dot(W.T, W), H), H)
cross_prod = trace_dot((X * H.T), W)
error = np.sqrt(norm_X + norm_WH - 2. * cross_prod)

In [80]:
error/X.shape[0]/X.shape[1]

0.00012841350131919519

In [29]:
def scorer(estimator, X, y=None):
    print(X.shape)
    return 0

In [22]:
class Estimator:
    def fit(X, y=None):
        pass

In [52]:
len(np.unique(ratings['MovieID']))

3706

In [60]:
np.sum(np.unique(ratings['UserID']) != np.array(range(1, 6041)))

0

In [64]:
len(set(range(1, 3953)) - set(ratings['MovieID']))

246

In [61]:
np.where(np.unique(ratings['MovieID']) - np.array(range(1, 3953)))

ValueError: operands could not be broadcast together with shapes (3706,) (3952,) 

In [66]:
np.unique(ratings['MovieID'])

array([   1,    2,    3, ..., 3950, 3951, 3952])

In [41]:
from sklearn.cross_validation import KFold
from scipy.sparse import csr_matrix

# ratings_array array of shape (n_ratings, 3)
n_ratings = ratings.shape[0]

cv = KFold(n_ratings)
cv = list(cv)
n_train_max = len(cv[0][0])

train_ratios = np.linspace(0, 1, 10)[1:]
train_sizes = [int(s * n_train_max) for s in train_ratios]

row = ratings['MovieID'].values
col = ratings['UserID'].values
data = ratings['Rating'].values

MSE = np.zeros((len(cv), len(train_sizes)))
for cv_i, (train_index, test_index) in enumerate(cv):
    square_error = 0
    for train_i, train_size in enumerate(train_sizes):
        train_index_actual = train_index[:train_size]
        train_data = csr_matrix((data[train_index_actual], (row[train_index_actual], col[train_index_actual])))

        W = model.fit_transform(train_data) 
        H = model.components_
        ratings_fit_trans = np.dot(W, H)
        print(ratings_fit_trans.shape)
        
        square_error = 0
        for i, j, r in zip(row[test_index], col[test_index], data[test_index]):
            rating_predicted = ratings_fit_trans[i, j]
            square_error += (rating_predicted - r)**2
        MSE[cv_i, train_i] = (square_error / len(test_index))**.5
        print(MSE)
MSE

(3953, 2447)
[[ 3.73029658  0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.        ]]
(3953, 2963)


KeyboardInterrupt: 

In [30]:
from sklearn.learning_curve import learning_curve
model = NMF(n_components=5, init='random', random_state=0)
lc = learning_curve(model, X, y = None, scoring=scorer, n_jobs=1, verbose=1)


[learning_curve] Training set sizes: [ 263  856 1449 2042 2635]
(1318, 6041)
(263, 6041)
(1318, 6041)
(856, 6041)
(1318, 6041)
(1449, 6041)
(1318, 6041)
(2042, 6041)
(1318, 6041)
(2635, 6041)
(1318, 6041)
(263, 6041)
(1318, 6041)
(856, 6041)
(1318, 6041)
(1449, 6041)
(1318, 6041)
(2042, 6041)
(1318, 6041)
(2635, 6041)
(1317, 6041)
(263, 6041)
(1317, 6041)
(856, 6041)
(1317, 6041)
(1449, 6041)
(1317, 6041)
(2042, 6041)
(1317, 6041)
(2635, 6041)


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   13.3s finished
